In [66]:
#!/usr/bin/env python3
import argparse
import numpy as np
from random import uniform as random
import random as rand
from math import pi as PI
from math import cos, sin, atan2, atan, tanh, fmod, exp, isclose
mag = np.hypot
import json
from itertools import takewhile
from namespace import Namespace
import time


should_write_training_data = False
should_read_training_data = False
BATCHES = 100
N_PLAYERS = 128
debug = False

ORDER = "C"


HALF_PI = PI / 2
TAU = PI * 2

g = 9.80665
drag_flat = 0.07
drag_narrow = 0.038
mass = 100
influence = 900

MUTATION_EFFECT = 0.20
MUTATION_CHANCE = 0.20
timedelta = 0.1

N_PARAMS = 11
TTL = 1000
OFFSET = (100, 100)
RANDOM_LOWER_BOUND = 15000
RANDOM_UPPER_BOUND = 15000
PARAM_LOWER_BOUND = -1
PARAM_UPPER_BOUND = 1
RANDOM_INITIAL = 15000
FITNESS_HYPERPARAMETER_WIDTH=10000
FITNESS_HYPERPARAMETER_HEIGHT=30000




In [67]:

def crossover(brain1, brain2):
    # bits are basically the binary choices for which parameter to take from which parent
    if hasattr(rand, "choices"):
        bits = rand.choices([0, 1], k=N_PARAMS)
    else:
        bits = [rand.choice([0, 1]) for _ in range(N_PARAMS)]
    new_brain = [[brain1[i], brain2[i]][bit] for i, bit in enumerate(bits)]
    return new_brain


def mutation(brain):
    for i, param in enumerate(brain):
        if random(0, 1) < MUTATION_CHANCE:
            # print("mutation occurred!")
            # brain.brain.params[i] = random(-MUTATION_EFFECT, MUTATION_EFFECT) + param
            brain[i] = rand.gauss(0, 2 * MUTATION_EFFECT) + param


def generate_children(brains, n=float("inf")):
    i = 0
    while i < n:
        brain1, brain2 = rand.sample(brains, 2)
        new_brain = crossover(brain1, brain2)
        mutation(new_brain)
        yield new_brain
        i += 1

PROPORTIONS = [0.25, 0.25, 0.6, 0.15]
def selection_crossover_and_breeding(fitnesses, brains):
    new_brains = []
    # sort by fitness, lower is better
    zipped = list(zip(list(fitnesses.T), list(brains.T)))
    zipped.sort(key=lambda e: e[0], reverse=True)
    brains = list(zip(*zipped))[1]
    # truncate X% worst players
    brains = brains[: int(PROPORTIONS[0]*N_PLAYERS)]
    # keep the Y% greatest brains
    new_brains.extend(brains[: int(PROPORTIONS[1]*N_PLAYERS)])

    # breed brains to fill Z% of the new population
    for new_brain in generate_children(brains, n=int(PROPORTIONS[2]*N_PLAYERS)):
        new_brains.append(new_brain)
    # fill the rest with new randoms
    while len(new_brains) < N_PLAYERS:
        new_brains.append(construct_brain())
    assert len(new_brains) == N_PLAYERS, (len(new_brains), N_PLAYERS)
    return np.array(new_brains, order=ORDER)

In [68]:



def sign(x):
    return 1 if x >= 0 else -1

def construct_brain():
    return [random(PARAM_LOWER_BOUND, PARAM_UPPER_BOUND) for _ in range(N_PARAMS)]

def direction(players):
    return np.arctan2(players[3], players[2]) + np.where(players[2] != 0, 0, (PI+HALF_PI))

def AoA(players):
    # return atan2(player[3], player[2])
    return np.mod(direction(players) - players[4], TAU)

def tangent(players):
#     return ((np.arctan2(-players[:, 2],players[:, 3]) + (PI if players[:, 3] > 0 else 0)) if players[:, 3] != 0 else HALF_PI) % TAU
    return np.mod(np.where(players[3] != 0, np.arctan2(-players[2],players[3]) + np.where(players[3] > 0, PI, 0), HALF_PI), TAU)


def pmag(players):
    return mag(players[2], players[3])


def lift_force(players):
    # angle = (self.theta - self.direction) % TAU

    y = 0.7 * 1.225 * 0.75 / (2*mass)
    # normal lift
    _AoA = AoA(players)
    _tan = tangent(players)
    mul = 50*y * pmag(players)**2 * np.cos(_AoA) * np.sin(_AoA)
    return [mul*np.cos(_tan), mul*np.sin(_tan)]

def simulate(players):
    L = lift_force(players)
    _mag = mag(players[2], players[3])
    # f = drag_narrow + (drag_flat-drag_narrow) * abs(sin(self.AoA))
    f = drag_narrow
    players[2] += (L[0] - f * _mag * players[2])/mass
    players[3] += (L[1] - g * mass - f * _mag * players[3])/mass
    players[0] += players[2] * timedelta
    players[1] -= players[3] * timedelta
    players[7] += timedelta

def update(players, brains):
    players[4] = evaluate(players, brains)

def construct_player(target):
    return np.array([
        0, # x
        0, # y
        0, # vx
        0, # vy
        0, # theta
        target[0], # target x
        target[1], # target y
        0, # time
        True, # alive
        0 # fitness
    ], dtype=np.float32)


def reset(players):
    players[0] = 0
    players[1] = 0
    players[2] = 0
    players[3] = 0
    players[4] = 0
    players[7] = 0
    players[-2] = True
    players[-1] = 0


# def construct_player_and_brain(target, params=None):
#     player = construct_player(target)
#     if params is None:
#         print("constructing brain")
#         brain = construct_brain()
#     else:
#         print("constructing brain from params")
#         brain = params[:]
#     update(player, brain) 
#     return player, brain

def construct_players_and_brains(DEST, filename=None):
    if should_read_training_data:
        print("reading training data")
        with open(filename, "r") as fd:
            data = json.load(fd)
        print(f"number of samples: {len(data['training_data'])}")

    players = []
    brains = []
    print("constructing brain from params")
    for i in range(N_PLAYERS):
        players.append(construct_player(DEST))
        brains.append(construct_brain())
    players, brains = np.array(players), np.array(brains)
    return np.array(players, order=ORDER).T, np.array(brains, order=ORDER).T

def transform_pos(x=0, y=0):
    """returns the screen coordinates to draw pos"""
    return vadd(OFFSET, (int(x / scale), int(y / scale)))


def reverse_transform_position(x, y):
    """returns the approximate real coordinates that correspond to screen coordinates"""
    return vsub([e*scale for e in [x,y]], OFFSET)


In [69]:


def out_of_bounds(players):
#     print(players)
    return np.where(players[0] < -100, True, False) | np.where(players[1] > players[6], True, False)
#     return np.

In [70]:


def batch_remap(variables):
#     print(variables.shape)
    return np.array([
        variables[0],
        variables[1],
        pmag(variables),
        direction(variables),
        variables[4],
        mag(variables[ 5] - variables[ 0], variables[ 6] - variables[ 1]),
        np.arctan2(variables[5] - variables[ 0], variables[ 6] - variables[ 1]),
        variables[5] - variables[0],
        variables[6] - variables[1],
        variables[7]
    ])

def strange_dot(l_of_params, l_of_remapped):
    return np.array(list(np.dot(v,p) for v,p in zip(list(l_of_params.T), list(l_of_remapped.T))))

def evaluate(l_of_variables, l_of_params):
    remapped = batch_remap(l_of_variables)
#     print(l_of_params.shape, remapped.shape)
    result = np.mod(
        l_of_params[-1] + strange_dot(l_of_params[:-1], remapped),
        TAU
    )
#     print(result.shape)
    return result

# def evaluate(l_of_variables, l_of_params):
#     new_l = []
#     for v,p in zip(list(l_of_variables), list(l_of_params)):
#         new_l.append(evaluate_for_one(v,p))
#     return new_l

In [71]:


def player_fitness_formula(players):
    return fitness_formula(players[ 0], players[ 1], players[5], players[ 6], players[ 7])

def fitness_formula(x, y, tx, ty, time):
    return FITNESS_HYPERPARAMETER_HEIGHT * np.exp(-(mag(tx-x, ty-y) / FITNESS_HYPERPARAMETER_WIDTH) ** 2) - 2*time

assert fitness_formula(0,0,0,0,0) == 30000


def main(read_file="savedata.json", write_file = "savedata.json"):




    FLOOR = 10000
    DEST = RANDOM_INITIAL, FLOOR


    players, brains = construct_players_and_brains(DEST, read_file)
    update(players, brains)
    assert len(players[0, :]) == len(brains[0, :])

    best_fitness = float("inf")

    for i in range(BATCHES):
        print(f"starting batch {i} of {BATCHES}")
        t1 = time.perf_counter()
        frame = 0
        halted = False



        alive_players_count = N_PLAYERS
        while not halted:
            if frame % 10 == 0:
                print(".", end='')
            simulate(players)
            
            update(players, brains)
            dead = np.bitwise_or(out_of_bounds(players), players[7] > TTL)
            players[ -2] = np.where(dead, 0, 1)
            players[ -1] = np.where(dead, player_fitness_formula(players), 0)
            alive_players_count = np.count_nonzero(players[ -2])
            if alive_players_count == 0:
                halted = True


            frame += 1
        t2 = time.perf_counter()
        best_fitness = max(*[e[-1] for e in players])
        new_target = random(RANDOM_LOWER_BOUND, RANDOM_UPPER_BOUND), FLOOR
        print(f"batch {i} of {BATCHES} = {round(100*i/BATCHES,2)}% done")
        print(f"best fitness was {best_fitness}")
        players[5] = new_target[0]
        players[6] = new_target[1]
        print("time for batch: {}".format(t2-t1))
        print(brains.shape)
        brains = selection_crossover_and_breeding(players[-1], brains).T
        print(brains.shape)
        # assert len(set(id(e) for e in brains)) == len(brains)
#         players = players[:len(brains)]
        reset(players)
        DEST = new_target
        t3 = time.perf_counter()
        print("time for resetting: {}".format(t3-t2))
        print("\n")



main()

constructing brain from params
starting batch 0 of 100
.........................................................................................batch 0 of 100 = 0.0% done
best fitness was 15000.0
time for batch: 0.3597213159991952
(11, 128)
(11, 128)
time for resetting: 0.0018611369996506255


starting batch 1 of 100
.........................................................................................batch 1 of 100 = 1.0% done
best fitness was 15000.0
time for batch: 0.36354855700119515
(11, 128)
(11, 128)
time for resetting: 0.0019338829988555517


starting batch 2 of 100
.........................................................................................batch 2 of 100 = 2.0% done
best fitness was 15000.0
time for batch: 0.37926483699993696
(11, 128)
(11, 128)
time for resetting: 0.002319969000382116


starting batch 3 of 100
..........................................................................................batch 3 of 100 = 3.0% done
best fitness was 15000.0
time for b

....................................................................................................................batch 28 of 100 = 28.0% done
best fitness was 15301.3203125
time for batch: 0.48918965499979095
(11, 128)
(11, 128)
time for resetting: 0.0020440799999050796


starting batch 29 of 100
....................................................................................................................batch 29 of 100 = 29.0% done
best fitness was 17365.447265625
time for batch: 0.49181522899925767
(11, 128)
(11, 128)
time for resetting: 0.0021590570013358956


starting batch 30 of 100
....................................................................................................................batch 30 of 100 = 30.0% done
best fitness was 18243.71484375
time for batch: 0.4694070469995495
(11, 128)
(11, 128)
time for resetting: 0.0024384410007769475


starting batch 31 of 100
...............................................................................................

............................................................................................................................batch 55 of 100 = 55.0% done
best fitness was 19414.828125
time for batch: 0.518532359999881
(11, 128)
(11, 128)
time for resetting: 0.00232051000057254


starting batch 56 of 100
............................................................................................................................batch 56 of 100 = 56.0% done
best fitness was 18836.37109375
time for batch: 0.5051439550006762
(11, 128)
(11, 128)
time for resetting: 0.001959831999556627


starting batch 57 of 100
............................................................................................................................batch 57 of 100 = 57.0% done
best fitness was 18712.5859375
time for batch: 0.5015782350001246
(11, 128)
(11, 128)
time for resetting: 0.0019069230002060067


starting batch 58 of 100
................................................................................

............................................................................................................................batch 82 of 100 = 82.0% done
best fitness was 16395.23046875
time for batch: 0.5115922850000061
(11, 128)
(11, 128)
time for resetting: 0.00231368700042367


starting batch 83 of 100
............................................................................................................................batch 83 of 100 = 83.0% done
best fitness was 18761.734375
time for batch: 0.5088192240000353
(11, 128)
(11, 128)
time for resetting: 0.00233085900072183


starting batch 84 of 100
............................................................................................................................batch 84 of 100 = 84.0% done
best fitness was 19452.513671875
time for batch: 0.5342938079993473
(11, 128)
(11, 128)
time for resetting: 0.0026426360000186833


starting batch 85 of 100
..............................................................................

In [64]:
test_players, test_brains = construct_players_and_brains((15000,10000))
simulate(test_players)
update(test_players, test_brains)

constructing brain from params


In [32]:
dead = np.bitwise_or(out_of_bounds(test_players), test_players[7] > TTL)

In [33]:
dead

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]

In [38]:
print(test_players)
test_players[-2] = np.where(dead, 0, 1)
test_players[-1] = np.where(dead, player_fitness_formula(test_players), 0)
print(test_players)

[[0.       0.       0.       ... 0.       0.       0.      ]
 [0.980665 0.980665 0.980665 ... 0.980665 0.980665 0.980665]
 [0.       0.       0.       ... 0.       0.       0.      ]
 ...
 [0.1      0.1      0.1      ... 0.1      0.1      0.1     ]
 [1.       1.       1.       ... 1.       1.       1.      ]
 [0.       0.       0.       ... 0.       0.       0.      ]]
[[0.       0.       0.       ... 0.       0.       0.      ]
 [0.980665 0.980665 0.980665 ... 0.980665 0.980665 0.980665]
 [0.       0.       0.       ... 0.       0.       0.      ]
 ...
 [0.1      0.1      0.1      ... 0.1      0.1      0.1     ]
 [1.       1.       1.       ... 1.       1.       1.      ]
 [0.       0.       0.       ... 0.       0.       0.      ]]


In [36]:
alive_players_count = np.count_nonzero(test_players[-2])
print(alive_players_count)

128


In [39]:
frame = 0
halted = False
while not halted:
    print(".", end='')
    simulate(test_players)

    update(test_players, test_brains)
    dead = np.bitwise_or(out_of_bounds(test_players), test_players[ 7] > TTL)
    test_players[ -2] = np.where(dead, 0, 1)
    test_players[ -1] = np.where(dead, player_fitness_formula(test_players), 0)
    alive_players_count = np.count_nonzero(test_players[-2])
    if alive_players_count == 0:
        halted = True


    frame += 1

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [44]:
test_players[5] = 15000
test_players[6] = 10000